In [8]:
import numpy as np
import pandas as pd
import re

In [82]:
atl = pd.read_csv('postings_atl.csv')
bos = pd.read_csv('postings_bos.csv')
chi = pd.read_csv('postings_chi.csv')
den = pd.read_csv('postings_den.csv')
la = pd.read_csv('postings_la.csv')
nyc = pd.read_csv('postings_nyc.csv')
sc = pd.read_csv('postings_sc.csv')
sea = pd.read_csv('postings_sea.csv')
sf = pd.read_csv('postings_sf.csv')


In [86]:
print(list(atl.columns.values))
print(list(bos.columns.values))
print(list(chi.columns.values))
print(list(den.columns.values))
print(list(la.columns.values))
print(list(nyc.columns.values))
print(list(sc.columns.values))
print(list(sea.columns.values))
print(list(sf.columns.values))

['ceo_approve', 'company', 'job_description', 'location', 'rating', 'recommend_friend', 'salary_est', 'title']
['ceo_approve', 'company', 'job_description', 'location', 'rating', 'recommend_friend', 'salary_est', 'title']
['ceo_approve', 'company', 'job_description', 'location', 'rating', 'recommend_friend', 'salary_est', 'title']
['ceo_approve', 'company', 'job_description', 'location', 'rating', 'recommend_friend', 'salary_est', 'title']
['ceo_approve', 'company', 'job_description', 'location', 'rating', 'recommend_friend', 'salary_est', 'title']
['ceo_approve', 'company', 'job_description', 'location', 'rating', 'recommend_friend', 'salary_est', 'title']
['ceo_approve', 'company', 'job_description', 'location', 'rating', 'recommend_friend', 'salary_est', 'title']
['ceo_approve', 'company', 'job_description', 'location', 'rating', 'recommend_friend', 'salary_est', 'title']
['ceo_approve', 'company', 'job_description', 'location', 'rating', 'recommend_friend', 'salary_est', 'title']


In [117]:
postings = pd.concat([atl, bos, chi, den, la, nyc, sc, sea, sf], axis = 0)

postings.shape

(7713, 8)

In [118]:
salary_high = []
for row in postings['salary_est']:
    try: 
        salary_high.append(row[-4:-1].strip('$'))
    except TypeError: 
        salary_high.append(np.nan)
        

In [119]:
postings['salary_low'] = postings['salary_est'].str.extract('(\d+)').astype(float)  
postings['salary_high'] = salary_high
postings['salary_high'] = postings['salary_high'].astype(float)
postings['salary_range'] = postings['salary_high'] - postings['salary_low']
postings['salary_mid'] = postings['salary_low'] + (postings['salary_range']/2)

postings.head()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  """Entry point for launching an IPython kernel.


,ceo_approve,company,job_description,location,rating,recommend_friend,salary_est,title,salary_low,salary_high,salary_range,salary_mid
0,77.0,SunTrust Banks –,"Job Description, PPNR Stress Analytics Finance...","Atlanta, GA",3.1,52.0,Est. Salary $69k-$100k,PPNR Stress Analytics Finance Manager,69.0,100.0,31.0,84.5
1,86.0,Careerbuilder –,"Job Description,Data Science Fall 2017 Interns...","Norcross, GA",3.7,68.0,NaN,Data Science Fall 2017 Internship - Recommenda...,NaN,NaN,NaN,NaN
2,98.0,SPH Analytics –,"Job Description, Data Analyst ,General Duties:...","Duluth, GA",2.8,49.0,Est. Salary $58k-$77k,Data Analyst,58.0,77.0,19.0,67.5
3,96.0,T-Mobile –,"Job Description, A creative development role o...","Atlanta, GA",3.9,82.0,NaN,"Developer, Software – Workforce Management Ana...",NaN,NaN,NaN,NaN
4,67.0,LiveHealthier –,"Job Description, Position Purpose: Responsible...","Smyrna, GA",2.8,55.0,Est. Salary $53k-$68k,Data Analyst IV,53.0,68.0,15.0,60.5


In [120]:
postings['company'] = postings['company'].str.replace(' – ', '')

In [121]:
state = []
for row in postings['location']:
    state.append(row[-2:])

In [122]:
city = []
for row in postings['location']:
    city.append(row[:-4])

In [123]:
postings['city'] = city
postings['state'] = state


In [124]:
postings = postings.drop('location', 1)
postings = postings.drop('salary_est', 1)

In [126]:
#postings = postings[np.isfinite(postings['ceo_approve'])]  # don't want to do this
postings.tail()

,ceo_approve,company,job_description,rating,recommend_friend,title,salary_low,salary_high,salary_range,salary_mid,city,state
987,36.0,Global Software Resources,"Job Description,Job Description – Data Scienti...",2.9,35.0,Data Scientist,127.0,187.0,60.0,157.0,San Francisco,CA
988,100.0,Avenue Code,"Job Description, Avenue Code is an eCommerce c...",4.3,96.0,Data Scientist,116.0,160.0,44.0,138.0,San Francisco,CA
989,100.0,Supercell,"Job Description, Description ,Food for thought...",4.4,81.0,Data Scientist,97.0,135.0,38.0,116.0,San Francisco,CA
990,88.0,The Climate Corp,"Job Description, Description ,Position Overvie...",3.9,76.0,Data Scientist,138.0,190.0,52.0,164.0,San Francisco,CA
991,-1.0,DataRobot,"Job Description,Job Summary, As a Customer Fac...",4.0,73.0,Partners - Customer Facing Data Scientist,97.0,135.0,38.0,116.0,San Francisco,CA


In [127]:
postings.to_csv('postings_master.csv', index=False)